<div style="text-align: center; line-height: 0; padding-top: 2px;">
  <img src="https://www.quantiaconsulting.com/logos/quantia_logo_orizz.png" alt="Quantia Consulting" style="width: 600px; height: 250px">
</div>

# Stream Ensemble Classification
---

## `NEWeather` dataset

**Description:** The National Oceanic and Atmospheric Administration (NOAA),
has compiled a database of weather measurements from over 7,000 weather 
stations worldwide. Records date back to the mid-1900s. Daily measurements
include a variety of features (temperature, pressure, wind speed, etc.) as
well as a series of indicators for precipitation and other weather-related
events. The `NEweather` dataset contains data from this database, specifically
from the Offutt Air Force Base in Bellevue, Nebraska ranging for over 50 years
(1949-1999).

**Features:** 8 Daily weather measurements
 
|       Attribute      | Description |
|:--------------------:|:-----------------------------|
| `temp`                   | Temperature
| `dew_pnt`                | Dew Point
| `sea_lvl_press`          | Sea Level Pressure
| `visibility`             | Visibility
| `avg_wind_spd`           | Average Wind Speed
| `max_sustained_wind_spd` | Maximum Sustained Wind Speed
| `max_temp`               | Maximum Temperature
| `min_temp`               | Minimum Temperature


**Class:** `rain` | 0: no rain, 1: rain
 
**Samples:** 18,159


In [1]:
import pandas as pd
from river.stream import iter_pandas
from river.metrics import Accuracy
from river.evaluate import progressive_val_score

In [2]:
data = pd.read_csv("../datasets/NEweather.csv")
features = data.columns[:-1]

## Online Bagging

---
[Online Bagging](https://riverml.xyz/latest/api/ensemble/BaggingClassifier/) is an Online bootstrap aggregation for classification.

In [3]:
from river.ensemble import BaggingClassifier
from river.tree import HoeffdingTreeClassifier

model = BaggingClassifier(model=HoeffdingTreeClassifier(),
                          n_models=10,
                          seed=42)
metric = Accuracy()
stream = iter_pandas(X=data[features], y=data['rain'])

progressive_val_score(dataset=stream,
                      model=model,
                      metric=metric,
                      print_every=1000)

[1,000] Accuracy: 71.17%
[2,000] Accuracy: 70.09%
[3,000] Accuracy: 70.89%
[4,000] Accuracy: 71.42%
[5,000] Accuracy: 72.07%
[6,000] Accuracy: 72.51%
[7,000] Accuracy: 73.38%
[8,000] Accuracy: 73.27%
[9,000] Accuracy: 73.36%
[10,000] Accuracy: 73.52%
[11,000] Accuracy: 73.99%
[12,000] Accuracy: 74.22%
[13,000] Accuracy: 74.31%
[14,000] Accuracy: 74.28%
[15,000] Accuracy: 74.54%
[16,000] Accuracy: 74.54%
[17,000] Accuracy: 74.61%
[18,000] Accuracy: 74.38%


Accuracy: 74.39%

## Leveraging Bagging
---

[Leveraging Bagging](https://riverml.xyz/latest/api/ensemble/LeveragingBaggingClassifier/) is an improvement over the Oza Bagging algorithm. The bagging performance is leveraged by increasing the re-sampling. It uses a poisson distribution to simulate the re-sampling process. To increase re-sampling it uses a higher w value of the Poisson distribution (agerage number of events), 6 by default, increasing the input space diversity, by attributing a different range of weights to the data samples.

To deal with concept drift, Leveraging Bagging uses the ADWIN algorithm to monitor the performance of each member of the enemble If concept drift is detected, the worst member of the ensemble (based on the error estimation by ADWIN) is replaced by a new (empty) classifier.

In [4]:
from river.ensemble import LeveragingBaggingClassifier
from river.tree import HoeffdingTreeClassifier

model = LeveragingBaggingClassifier(model=HoeffdingTreeClassifier(),
                          n_models=10,
                          seed=42)
metric = Accuracy()
stream = iter_pandas(X=data[features], y=data['rain'])

progressive_val_score(dataset=stream,
                      model=model,
                      metric=metric,
                      print_every=1000)

[1,000] Accuracy: 75.78%
[2,000] Accuracy: 76.64%
[3,000] Accuracy: 77.79%
[4,000] Accuracy: 77.79%
[5,000] Accuracy: 77.70%
[6,000] Accuracy: 77.80%
[7,000] Accuracy: 78.18%
[8,000] Accuracy: 77.78%
[9,000] Accuracy: 77.64%
[10,000] Accuracy: 77.83%
[11,000] Accuracy: 78.10%
[12,000] Accuracy: 78.21%
[13,000] Accuracy: 78.36%
[14,000] Accuracy: 78.17%
[15,000] Accuracy: 78.33%
[16,000] Accuracy: 78.38%
[17,000] Accuracy: 78.40%
[18,000] Accuracy: 78.14%


Accuracy: 78.15%

## AdaptiveRandomForest
---



The 3 most important aspects of [ARF](https://riverml.xyz/latest/api/ensemble/AdaptiveRandomForestClassifier/) are:
- inducing diversity through re-sampling
- inducing diversity through randomly selecting subsets of features for node splits
- drift detectors per base tree, which cause selective resets in response to drifts

It also allows training background trees, which start training if a warning is detected and replace the active tree if the warning escalates to a drift.

In [5]:
from river.ensemble import AdaptiveRandomForestClassifier

model = AdaptiveRandomForestClassifier(n_models=10)
metric = Accuracy()
stream = iter_pandas(X=data[features], y=data['rain'])

progressive_val_score(dataset=stream,
                      model=model,
                      metric=metric,
                      print_every=1000)

[1,000] Accuracy: 75.18%
[2,000] Accuracy: 76.84%
[3,000] Accuracy: 78.19%
[4,000] Accuracy: 77.94%
[5,000] Accuracy: 77.88%
[6,000] Accuracy: 78.16%
[7,000] Accuracy: 78.68%
[8,000] Accuracy: 78.20%
[9,000] Accuracy: 78.26%
[10,000] Accuracy: 78.28%
[11,000] Accuracy: 78.43%
[12,000] Accuracy: 78.59%
[13,000] Accuracy: 78.48%
[14,000] Accuracy: 78.19%
[15,000] Accuracy: 78.38%
[16,000] Accuracy: 78.39%
[17,000] Accuracy: 78.38%
[18,000] Accuracy: 78.18%


Accuracy: 78.17%

## StreamingRandomPatches
---
[SRP](https://riverml.xyz/latest/api/ensemble/SRPClassifier/) is an ensemble method that simulates bagging or random subspaces. The default algorithm uses both bagging and random subspaces, namely Random Patches. The default base estimator is a Hoeffding Tree, but other base estimators can be used (differently from random forest variations).

In [6]:
from river.ensemble import SRPClassifier
from river.tree import HoeffdingTreeClassifier

model = SRPClassifier(model=HoeffdingTreeClassifier(),
                      n_models=10,
                      seed=42)
metric = Accuracy()
stream = iter_pandas(X=data[features], y=data['rain'])

progressive_val_score(dataset=stream, 
                      model=model, 
                      metric=metric, 
                      print_every=1000)

[1,000] Accuracy: 73.57%
[2,000] Accuracy: 76.29%
[3,000] Accuracy: 77.79%
[4,000] Accuracy: 78.12%
[5,000] Accuracy: 78.08%
[6,000] Accuracy: 77.98%
[7,000] Accuracy: 78.47%
[8,000] Accuracy: 78.16%
[9,000] Accuracy: 78.16%
[10,000] Accuracy: 78.35%
[11,000] Accuracy: 78.66%
[12,000] Accuracy: 78.78%
[13,000] Accuracy: 78.81%
[14,000] Accuracy: 78.56%
[15,000] Accuracy: 78.67%
[16,000] Accuracy: 78.68%
[17,000] Accuracy: 78.73%
[18,000] Accuracy: 78.53%


Accuracy: 78.60%

## Concept Drift Impact

Concept drift can negatively impact learning methods if not properly handled. Multiple real-world applications suffer **model degradation** as the models can not adapt to changes in the data.

---
## `AGRAWAL` dataset

We will load the data from a csv file. The data was generated using the `AGRAWAL` data generator with 3 **gradual drifts** at the 5k, 10k, and 15k marks. It contains 9 features, 6 numeric and 3 categorical.

There are 10 functions for generating binary class labels from the features. These functions determine whether a **loan** should be approved.

| Feature    | Description            | Values                                                                |
|------------|------------------------|-----------------------------------------------------------------------|
| `salary`     | salary                 | uniformly distributed from 20k to 150k                                |
| `commission` | commission             | if (salary <   75k) then 0 else uniformly distributed from 10k to 75k |
| `age`        | age                    | uniformly distributed from 20 to 80                                   |
| `elevel`     | education level        | uniformly chosen from 0 to 4                                          |
| `car`        | car maker              | uniformly chosen from 1 to 20                                         |
| `zipcode`    | zip code of the town   | uniformly chosen from 0 to 8                                          |
| `hvalue`     | value of the house     | uniformly distributed from 50k x zipcode to 100k x zipcode            |
| `hyears`     | years house owned      | uniformly distributed from 1 to 30                                    |
| `loan`       | total loan amount      | uniformly distributed from 0 to 500k                                  |

**Class:** `y` | 0: no loan, 1: loan
 
**Samples:** 20,000

`elevel`, `car`, and `zipcode` are categorical features.

In [7]:
data = pd.read_csv("../datasets/agr_a_20k.csv")
features = data.columns[:-1]

## ADWIN Online Bagging

---
[ADWIN Online Bagging](https://riverml.xyz/latest/api/ensemble/ADWINBaggingClassifier/) is the online bagging method with the addition of the ADWIN algorithm as a change detector. If concept drift is detected, the worst member of the ensemble (based on the error estimation by ADWIN) is replaced by a new (empty) classifier.

In [9]:
from river.ensemble import ADWINBaggingClassifier
from river.tree import HoeffdingTreeClassifier

model = ADWINBaggingClassifier(model=HoeffdingTreeClassifier(nominal_attributes=['elevel', 'car', 'zipcode']),
                          n_models=10,
                          seed=42)
metric = Accuracy()
stream = iter_pandas(X=data[features], y=data['class'])

progressive_val_score(dataset=stream,
                      model=model,
                      metric=metric,
                      print_every=1000)

[1,000] Accuracy: 80.18%
[2,000] Accuracy: 84.04%
[3,000] Accuracy: 85.46%
[4,000] Accuracy: 86.70%
[5,000] Accuracy: 87.62%
[6,000] Accuracy: 83.00%
[7,000] Accuracy: 81.34%
[8,000] Accuracy: 80.25%
[9,000] Accuracy: 79.06%
[10,000] Accuracy: 78.13%
[11,000] Accuracy: 77.10%
[12,000] Accuracy: 77.38%
[13,000] Accuracy: 77.68%
[14,000] Accuracy: 78.28%
[15,000] Accuracy: 78.98%
[16,000] Accuracy: 77.80%
[17,000] Accuracy: 77.41%
[18,000] Accuracy: 77.15%
[19,000] Accuracy: 76.89%
[20,000] Accuracy: 76.60%


Accuracy: 76.60%

## Leveraging Bagging

In [10]:
from river.ensemble import LeveragingBaggingClassifier
from river.tree import HoeffdingTreeClassifier

model = LeveragingBaggingClassifier(model=HoeffdingTreeClassifier(nominal_attributes=['elevel', 'car', 'zipcode']),
                          n_models=10,
                          seed=42)
metric = Accuracy()
stream = iter_pandas(X=data[features], y=data['class'])

progressive_val_score(dataset=stream,
                      model=model,
                      metric=metric,
                      print_every=1000)

[1,000] Accuracy: 75.48%
[2,000] Accuracy: 79.34%
[3,000] Accuracy: 81.83%
[4,000] Accuracy: 83.45%
[5,000] Accuracy: 84.56%
[6,000] Accuracy: 80.01%
[7,000] Accuracy: 77.38%
[8,000] Accuracy: 75.73%
[9,000] Accuracy: 74.85%
[10,000] Accuracy: 73.91%
[11,000] Accuracy: 72.83%
[12,000] Accuracy: 73.62%
[13,000] Accuracy: 74.35%
[14,000] Accuracy: 75.34%
[15,000] Accuracy: 76.15%
[16,000] Accuracy: 75.10%
[17,000] Accuracy: 74.69%
[18,000] Accuracy: 74.42%
[19,000] Accuracy: 74.30%
[20,000] Accuracy: 73.97%


Accuracy: 73.97%

## AdaptiveRandomForest

In [11]:
from river.ensemble import AdaptiveRandomForestClassifier

model = AdaptiveRandomForestClassifier(n_models=10,nominal_attributes=['elevel', 'car', 'zipcode'])
metric = Accuracy()
stream = iter_pandas(X=data[features], y=data['class'])

progressive_val_score(dataset=stream,
                      model=model,
                      metric=metric,
                      print_every=1000)

[1,000] Accuracy: 78.78%
[2,000] Accuracy: 80.19%
[3,000] Accuracy: 80.69%
[4,000] Accuracy: 81.77%
[5,000] Accuracy: 82.20%
[6,000] Accuracy: 77.26%
[7,000] Accuracy: 75.65%
[8,000] Accuracy: 74.56%
[9,000] Accuracy: 73.89%
[10,000] Accuracy: 73.17%
[11,000] Accuracy: 71.34%
[12,000] Accuracy: 71.48%
[13,000] Accuracy: 71.92%
[14,000] Accuracy: 72.61%
[15,000] Accuracy: 73.04%
[16,000] Accuracy: 72.12%
[17,000] Accuracy: 72.00%
[18,000] Accuracy: 72.13%
[19,000] Accuracy: 72.31%
[20,000] Accuracy: 72.35%


Accuracy: 72.35%

## StreamingRandomPatches
---
We set the drift and warning detection options

In [12]:
from river.ensemble import SRPClassifier
from river.tree import HoeffdingTreeClassifier
from river.drift import ADWIN

model = SRPClassifier(model=HoeffdingTreeClassifier(nominal_attributes=['elevel', 'car', 'zipcode']),
                      n_models=10,
                      drift_detector=ADWIN(delta=0.001),
                      warning_detector=ADWIN(delta=0.01),
                      seed=42)
metric = Accuracy()
stream = iter_pandas(X=data[features], y=data['class'])

progressive_val_score(dataset=stream, 
                      model=model, 
                      metric=metric, 
                      print_every=1000)

[1,000] Accuracy: 90.59%
[2,000] Accuracy: 92.35%
[3,000] Accuracy: 92.90%
[4,000] Accuracy: 93.17%
[5,000] Accuracy: 93.34%
[6,000] Accuracy: 87.96%
[7,000] Accuracy: 84.54%
[8,000] Accuracy: 82.45%
[9,000] Accuracy: 81.08%
[10,000] Accuracy: 79.79%
[11,000] Accuracy: 78.73%
[12,000] Accuracy: 79.63%
[13,000] Accuracy: 80.60%
[14,000] Accuracy: 81.63%
[15,000] Accuracy: 82.46%
[16,000] Accuracy: 81.58%
[17,000] Accuracy: 81.38%
[18,000] Accuracy: 81.08%
[19,000] Accuracy: 80.91%
[20,000] Accuracy: 80.60%


Accuracy: 80.60%

##### ![Quantia Tiny Logo](https://www.quantiaconsulting.com/logos/quantia_logo_tiny.png) Quantia Consulting, srl. All rights reserved.